In [1]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import pickle

In [2]:
def get_infos(house_soup):
    house_info = dict()
    house_info['Location'] = house_soup.find(class_ = "single_title").find_all("span")[2].get_text()
    house_info['Price'] = house_soup.find(class_="price_icon").get_text()[16:-9]
    house_info['Area'] = house_soup.find(class_="size_icon").get_text()[25:-11]
    return house_info

In [3]:
page = 1
house_infos = list()
while True:
    url = requests.get("https://viraland.vn/?s&loai-tin=cho-thue&danh-muc-tin=cho-thue-phong-tro&tinh=Th%C3%A0nh+ph%E1%BB%91+%C4%90%C3%A0+N%E1%BA%B5ng&huyen&dien-tich&gia&huong&fbclid=IwAR3Z4m8PeK9ZhvbRzXxhfTuo88G6ZUzQFVvZujiZGPxPj3CtvmgRmzbIntA&paged=" + str(page))
    page_soup = bs(url.text, 'html.parser')
    if page_soup.find(class_="tin_rao_title").get_text() == "\n0 kết quả được tìm thấy\n": 
        break
    houses = page_soup.body.section.div.div.div.div.find_all("a", href=True)
    if (houses == []): break
    for idx in range(len(houses)):
        houses[idx] = houses[idx]['href']
    houses = list(set(houses))
    for house in houses:
        try:
            house_page = requests.get(house)
            house_soup = bs(house_page.text, 'html.parser')
        except: continue
        try: 
            house_infos.append(get_infos(house_soup))
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/viraland.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
if (house_infos):
    with open('../Raw_Data/viraland.pickle','wb') as handle:
        pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
with open("../Raw_Data/viraland.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset


,Location,Price,Area
0,"Cho thuê phòng trọ đường Lý Văn Phức, P. Hòa H...",2 triệu,KXĐ
1,"Cho thuê phòng trọ đường Trưng Nữ Vương, P. Hò...","1,4 triệu",KXĐ
2,"Cho thuê phòng trọ đường Hà Thị Thân, An Hải T...",Thoả thuận,KXĐ
3,"Cho thuê phòng trọ đường Vũ Miên, Hoà Phước, H...",Thoả thuận,25 m2
4,"Cho thuê phòng trọ đường Trương Định, Mân Thái...",3 triệu,35 m2
...,...,...,...
1426,"Cho thuê phòng trọ đường Trần Cao Vân, Thanh K...","2,5 triệu",16 m2
1427,"Cho thuê trọ đường Nguyễn Thái Bình, Hòa Minh,...","2,8 triệu",30 m2
1428,"Cho thuê trọ đường Phạm Văn Tráng, Hòa Khánh B...",2 triệu,100 m2
1429,"Cho thuê phòng trọ đường Núi Thành, Hòa Cường ...","1,05 triệu",35 m2


In [5]:
dataset.to_csv('../Raw_Data/viraland.csv')